## Importing Libraries

In [ ]:
import mysql.connector
from mysql.connector import Error
from typing import Optional, List, Any, Dict
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
import psycopg2

## Initial CSV data

In [10]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## DB Class

In [28]:
load_dotenv()
class ChurnDatabaseSetup:
    def __init__(self, host: str = "localhost",
                 user: str = "root",
                 password: str = os.getenv('MYSQL_PASS'),
                 port: int = 3306):
        self.connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            port=port
        )
        self.cursor = self.connection.cursor()
        
    def create_database(self):
        try:
            # Create database
            self.cursor.execute("CREATE DATABASE IF NOT EXISTS churn")
            self.cursor.execute("USE churn")
            
            # Create tables
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS customers (
                    customer_id VARCHAR(50) PRIMARY KEY,
                    gender VARCHAR(10),
                    is_senior_citizen BOOLEAN,
                    has_partner BOOLEAN,
                    has_dependents BOOLEAN,
                    tenure INT,
                    monthly_charges DECIMAL(10,2),
                    total_charges DECIMAL(10,2)
                )
            """)
            
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS service_types (
                    service_type_id INT AUTO_INCREMENT PRIMARY KEY,
                    service_name VARCHAR(50) UNIQUE
                )
            """)
            
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS customer_services (
                    customer_id VARCHAR(50),
                    service_type_id INT,
                    status VARCHAR(50),
                    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
                    FOREIGN KEY (service_type_id) REFERENCES service_types(service_type_id),
                    PRIMARY KEY (customer_id, service_type_id)
                )
            """)
            
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS contract_info (
                    customer_id VARCHAR(50) PRIMARY KEY,
                    contract_type VARCHAR(50),
                    payment_method VARCHAR(50),
                    is_paperless_billing BOOLEAN,
                    has_churned BOOLEAN,
                    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
                )
            """)
            
            # Insert service types
            service_types = [
                'PhoneService',
                'MultipleLines',
                'InternetService',
                'OnlineSecurity',
                'OnlineBackup',
                'DeviceProtection',
                'TechSupport',
                'StreamingTV',
                'StreamingMovies'
            ]
            
            for service in service_types:
                self.cursor.execute("""
                    INSERT IGNORE INTO service_types (service_name)
                    VALUES (%s)
                """, (service,))
                
            self.connection.commit()
            
        except Error as e:
            print(f"Error creating database: {e}")
            self.connection.rollback()
            raise

    def select_database(self, dbname):
        self.cursor.execute(f"USE {dbname}")
        return
        
    def insert_data(self, df: pd.DataFrame):
        try:
            # Insert customer data
            for _, row in df.iterrows():
                # Insert into customers table
                self.cursor.execute("""
                    INSERT INTO customers 
                    (customer_id, gender, is_senior_citizen, has_partner, 
                     has_dependents, tenure, monthly_charges, total_charges)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    row['customerID'],
                    row['gender'],
                    bool(row['SeniorCitizen']),
                    row['Partner'] == 'Yes',
                    row['Dependents'] == 'Yes',
                    row['tenure'],
                    row['MonthlyCharges'],
                    float(row['TotalCharges']) if row['TotalCharges'].strip() else 0.0
                ))
                
                # Insert contract info
                self.cursor.execute("""
                    INSERT INTO contract_info 
                    (customer_id, contract_type, payment_method, 
                     is_paperless_billing, has_churned)
                    VALUES (%s, %s, %s, %s, %s)
                """, (
                    row['customerID'],
                    row['Contract'],
                    row['PaymentMethod'],
                    row['PaperlessBilling'] == 'Yes',
                    row['Churn'] == 'Yes'
                ))
                
                # Get service type mappings
                self.cursor.execute("SELECT service_type_id, service_name FROM service_types")
                service_map = {name: id for id, name in self.cursor.fetchall()}
                
                # Insert service statuses
                services_data = [
                    ('PhoneService', row['PhoneService']),
                    ('MultipleLines', row['MultipleLines']),
                    ('InternetService', row['InternetService']),
                    ('OnlineSecurity', row['OnlineSecurity']),
                    ('OnlineBackup', row['OnlineBackup']),
                    ('DeviceProtection', row['DeviceProtection']),
                    ('TechSupport', row['TechSupport']),
                    ('StreamingTV', row['StreamingTV']),
                    ('StreamingMovies', row['StreamingMovies'])
                ]
                
                for service_name, status in services_data:
                    self.cursor.execute("""
                        INSERT INTO customer_services 
                        (customer_id, service_type_id, status)
                        VALUES (%s, %s, %s)
                    """, (
                        row['customerID'],
                        service_map[service_name],
                        status
                    ))
            
            self.connection.commit()
            
        except Error as e:
            print(f"Error inserting data: {e}")
            self.connection.rollback()
            raise
            
    def show_customer_interval(self, 
                             start_date: datetime = datetime.now() - timedelta(days=365),
                             end_date: datetime = datetime.now()) -> List[Dict[str, Any]]:
        try:
            query = """
                SELECT 
                    c.customer_id,
                    c.gender,
                    c.tenure,
                    c.monthly_charges,
                    c.total_charges,
                    ci.contract_type,
                    ci.payment_method,
                    ci.has_churned,
                    DATE_ADD(CURDATE(), INTERVAL -c.tenure MONTH) as start_date
                FROM 
                    customers c
                JOIN 
                    contract_info ci ON c.customer_id = ci.customer_id
                WHERE 
                    DATE_ADD(CURDATE(), INTERVAL -c.tenure MONTH) 
                    BETWEEN %s AND %s
                ORDER BY 
                    start_date DESC
            """
            
            self.cursor.execute(query, (start_date, end_date))
            columns = [desc[0] for desc in self.cursor.description]
            results = [dict(zip(columns, row)) for row in self.cursor.fetchall()]
            
            return results
            
        except Error as e:
            print(f"Error querying customer interval: {e}")
            return []

    def show_customer_tenure(self, min_tenure: int = 0) -> List[Dict[str, Any]]:
        try:
            query = """
                SELECT 
                    c.customer_id,
                    c.gender,
                    c.tenure,
                    c.monthly_charges,
                    c.total_charges,
                    ci.contract_type,
                    ci.payment_method,
                    ci.has_churned,
                    GROUP_CONCAT(
                        CONCAT(st.service_name, ': ', cs.status)
                        SEPARATOR '; '
                    ) as services
                FROM 
                    customers c
                JOIN 
                    contract_info ci ON c.customer_id = ci.customer_id
                LEFT JOIN 
                    customer_services cs ON c.customer_id = cs.customer_id
                LEFT JOIN 
                    service_types st ON cs.service_type_id = st.service_type_id
                WHERE 
                    c.tenure >= %s
                GROUP BY 
                    c.customer_id
                ORDER BY 
                    c.tenure DESC
            """
            
            self.cursor.execute(query, (min_tenure,))
            columns = [desc[0] for desc in self.cursor.description]
            results = [dict(zip(columns, row)) for row in self.cursor.fetchall()]
            
            return results
            
        except Error as e:
            print(f"Error querying customer tenure: {e}")
            return []

    def show_customer_with_partner(self, partner: bool = True) -> List[Dict[str, Any]]:
        try:
            query = """
                SELECT 
                    c.customer_id,
                    c.gender,
                    c.tenure,
                    c.has_dependents,
                    c.monthly_charges,
                    c.total_charges,
                    ci.contract_type,
                    ci.payment_method,
                    ci.has_churned,
                    GROUP_CONCAT(
                        CONCAT(st.service_name, ': ', cs.status)
                        SEPARATOR '; '
                    ) as services
                FROM 
                    customers c
                JOIN 
                    contract_info ci ON c.customer_id = ci.customer_id
                LEFT JOIN 
                    customer_services cs ON c.customer_id = cs.customer_id
                LEFT JOIN 
                    service_types st ON cs.service_type_id = st.service_type_id
                WHERE 
                    c.has_partner = %s
                GROUP BY 
                    c.customer_id
                ORDER BY 
                    c.tenure DESC
            """
            
            self.cursor.execute(query, (partner,))
            columns = [desc[0] for desc in self.cursor.description]
            results = [dict(zip(columns, row)) for row in self.cursor.fetchall()]
            
            return results
            
        except Error as e:
            print(f"Error querying customers by partner status: {e}")
            return []
        
    
    def close(self):
        if hasattr(self, 'cursor') and self.cursor:
            self.cursor.close()
        if hasattr(self, 'connection') and self.connection:
            self.connection.close()

## Loading Initial Data

In [21]:
try:
    db_setup = ChurnDatabaseSetup()
    db_setup.create_database()

    db_setup.insert_data(df)
    
finally:
    db_setup.close()


In [29]:
try:
    db = ChurnDatabaseSetup()
    db.select_database("churn")
    
    recent_customers = db.show_customer_interval(
        start_date=datetime.now() - timedelta(days=90) # Get customers from last 3 months
    )
    
    loyal_customers = db.show_customer_tenure(min_tenure=24) # Get loyal customers (>= 2 years)
    
    
    single_customers = db.show_customer_with_partner(partner=False) # Get customers without partners
    
    for customer in recent_customers[:5]:
        print(f"Customer ID: {customer['customer_id']}")
        print(f"Tenure: {customer['tenure']} months")
        print(f"Contract: {customer['contract_type']}")
        print("---")

finally:
    db.close()

Customer ID: 1371-DWPAZ
Tenure: 0 months
Contract: Two year
---
Customer ID: 2520-SGTTA
Tenure: 0 months
Contract: Two year
---
Customer ID: 2775-SEFEE
Tenure: 0 months
Contract: Two year
---
Customer ID: 2923-ARZLG
Tenure: 0 months
Contract: One year
---
Customer ID: 3115-CZMZD
Tenure: 0 months
Contract: Two year
---
